In [4]:
import numpy as np
import pandas as pd
import os

In [17]:
file1=os.path.expanduser('~/Downloads/lab/log/2019-01-02T14-46-24/logstream_Ser.bin')
file2=os.path.expanduser('~/Downloads/lab/log/2019-01-02T14-50-31/logstream_Ser.bin')

In [18]:
# Create a dtype with the binary data format and the desired column names
dt = np.dtype([('idx', '>u1'),
               ('cmdSec', '>u4'), ('cmdUsec', '>u4'), ('cmd', '>u4'),
               ('fbSec', '>u4'), ('fbUsec', '>u4'),
               
               ('length', '>u2'),
               ('counts', '>u2'),
               ('voltage', '>u2'),
               ('temperature', '>u2'),
               ('current', '>u2'),
               ('id', 'u1')])

In [19]:
data1 = np.fromfile(file1, dtype=dt)
df1 = pd.DataFrame(data1)

data2 = np.fromfile(file2, dtype=dt)
df2 = pd.DataFrame(data2)

In [21]:
df2

,idx,cmdSec,cmdUsec,cmd,fbSec,fbUsec,length,counts,voltage,temperature,current,id
0,0,3202,32097,0,3202,32347,9,0,32128,40896,0,171
1,0,3202,33053,0,3202,33280,9,0,32128,40896,0,171
2,0,3202,34076,0,3202,34304,9,0,32128,40896,0,171
3,0,3202,34985,0,3202,35217,9,0,32128,40896,0,171
4,0,3202,35953,0,3202,36185,9,0,32128,40896,0,171
5,0,3202,36924,0,3202,37122,9,0,32128,40896,0,171
6,0,3202,37890,0,3202,38087,9,0,32128,40896,0,171
7,0,3202,38859,0,3202,39056,9,0,32128,40896,0,171
8,0,3202,39828,0,3202,40025,9,0,32128,40896,0,171
9,0,3202,40801,0,3202,41001,9,0,32128,40896,0,171
